##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Otimizadores do TensorFlow Addons: CyclicalLearningRate

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/addons/tutorials/optimizers_cyclicallearningrate"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/optimizers_cyclicallearningrate.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/addons/tutorials/optimizers_cyclicallearningrate.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/addons/tutorials/optimizers_cyclicallearningrate.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Este tutorial demonstra o uso do Cyclical Learning Rate do pacote do Addons.

## Cyclical Learning Rates

Mostrou-se vantajoso ajustar a taxa de aprendizado conforme o treinamento avança para uma rede neural. Isso tem inúmeros benefícios, desde a recuperação de ponto de sela até a prevenção de instabilidades numéricas que podem surgir durante a retropropagação. Mas como alguém sabe quanto ajustar em relação a um timestamp de treinamento específico? Em 2015, Leslie Smith percebeu que você deve aumentar a taxa de aprendizado para atravessar mais rápido o cenário de perda, mas também reduzir a taxa de aprendizado ao se aproximar da convergência. Para concretizar essa ideia, ele propôs as [Cyclical Learning Rates](https://arxiv.org/abs/1506.01186) (CLR), em que você ajusta a taxa de aprendizado em relação aos ciclos de uma função. Para uma demonstração visual, confira [este blog](https://www.jeremyjordan.me/nn-learning-rate/). A CLR está agora disponível como uma API do TensorFlow. Para mais detalhes, confira o artigo original [aqui](https://arxiv.org/abs/1506.01186). 

## Configuração

In [ ]:
!pip install -q -U tensorflow_addons

In [ ]:
from tensorflow.keras import layers
import tensorflow_addons as tfa
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

tf.random.set_seed(42)
np.random.seed(42)

## Carregue e prepare o dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

## Defina hiperparâmetros

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10
INIT_LR = 1e-4
MAX_LR = 1e-2

## Defina os utilitários de criação e treinamento do modelo

In [ ]:
def get_training_model():
    model = tf.keras.Sequential(
        [
            layers.InputLayer((28, 28, 1)),
            layers.experimental.preprocessing.Rescaling(scale=1./255),
            layers.Conv2D(16, (5, 5), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(32, (5, 5), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.SpatialDropout2D(0.2),
            layers.GlobalAvgPool2D(),
            layers.Dense(128, activation="relu"),
            layers.Dense(10, activation="softmax"),
        ]
    )
    return model

def train_model(model, optimizer):
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                       metrics=["accuracy"])
    history = model.fit(x_train,
        y_train,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        epochs=EPOCHS)
    return history

Considerando a reprodutibilidade, os pesos iniciais do modelo são serializados e serão usados para conduzir nossos experimentos. 

In [ ]:
initial_model = get_training_model()
initial_model.save("initial_model")

## Treine um modelo sem CLR

In [ ]:
standard_model = tf.keras.models.load_model("initial_model")
no_clr_history = train_model(standard_model, optimizer="sgd")

## Defina a programação da CLR

O módulo `tfa.optimizers.CyclicalLearningRate` retorna uma programação direta que pode ser passada para um otimizador. A programação aceita um passo como entrada e gera um valor calculado usando a fórmula CLR conforme descrita no artigo. 

In [ ]:
steps_per_epoch = len(x_train) // BATCH_SIZE
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=INIT_LR,
    maximal_learning_rate=MAX_LR,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * steps_per_epoch
)
optimizer = tf.keras.optimizers.SGD(clr)

Aqui, você especifica os limites inferior e superior da taxa de aprendizado, e a programação *oscilará* entre esse intervalo ([1e-4, 1e-2] nesse caso). `scale_fn` é usado para definir a função que ampliaria e reduziria a taxa de aprendizado em um determinado ciclo. `step_size` define a duração de um único ciclo. Um `step_size` de 2 significa que você precisa de um total de 4 iterações para concluir um ciclo. O valor recomendado para `step_size` é o seguinte:

`factor * steps_per_epoch`, em que o fator está dentro do intervalo [2, 8]. 

No mesmo [artigo da CLR](https://arxiv.org/abs/1506.01186), Leslie também apresentou um método simples e elegante para escolher os limites da taxa de aprendizado. Recomendamos que você o confira também. [Esta postagem de blog](https://www.pyimagesearch.com/2019/08/05/keras-learning-rate-finder/) fornece uma ótima introdução ao método.

Abaixo, visualize a programação `clr`. 

In [ ]:
step = np.arange(0, EPOCHS * steps_per_epoch)
lr = clr(step)
plt.plot(step, lr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.show()

Para visualizar melhor o efeito da CLR, você pode plotar a programação com um número maior de passos. 

In [ ]:
step = np.arange(0, 100 * steps_per_epoch)
lr = clr(step)
plt.plot(step, lr)
plt.xlabel("Steps")
plt.ylabel("Learning Rate")
plt.show()

A função que você está usando neste tutorial se refere ao método `triangular2` no artigo CLR. Há outras duas funções que foram exploradas, especificamente `triangular` e `exp` (abreviação de exponencial). 

## Treine um modelo com CLR

In [ ]:
clr_model = tf.keras.models.load_model("initial_model")
clr_history = train_model(clr_model, optimizer=optimizer)

Conforme esperado, a perda começa mais alta do que o normal e depois estabiliza com o avanço dos ciclos. Você pode confirmar isso visualmente com os plots abaixo. 

## Visualize as perdas

In [ ]:
(fig, ax) = plt.subplots(2, 1, figsize=(10, 8))

ax[0].plot(no_clr_history.history["loss"], label="train_loss")
ax[0].plot(no_clr_history.history["val_loss"], label="val_loss")
ax[0].set_title("No CLR")
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Loss")
ax[0].set_ylim([0, 2.5])
ax[0].legend()

ax[1].plot(clr_history.history["loss"], label="train_loss")
ax[1].plot(clr_history.history["val_loss"], label="val_loss")
ax[1].set_title("CLR")
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Loss")
ax[1].set_ylim([0, 2.5])
ax[1].legend()

fig.tight_layout(pad=3.0)
fig.show()

Embora você não tenha visto muito os efeitos da CLR neste exemplo de brinquedo, é um dos principais ingredientes por trás da [Superconvergência](https://arxiv.org/abs/1708.07120) e pode ter um [impacto excelente](https://www.fast.ai/2018/08/10/fastai-diu-imagenet/) ao treinar em cenários de grande escala. 